In [37]:
!ln -s /kaggle/input/caltech-birds-2011-dataset/CUB_200_2011/images ./filelists/CUB/images
!ln -s /kaggle/input/miniimagenet/miniImageNet/images ./filelists/miniImagenet/images
!ln -s /kaggle/input/d/arjun2000ashok/tieredimagenet/tiered_imagenet/ ./filelists/tieredImagenet/images
!ln -s /kaggle/input/stanford-dogs-dataset/images/Images ./filelists/dogs/Images
!ln -s /kaggle/input/stanford-cars-dataset/cars_train/cars_train ./filelists/cars/images
!ln -s /kaggle/input/fgvc-aircraft/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/images ./filelists/aircrafts/images

ln: failed to create symbolic link './filelists/CUB/images/images': Read-only file system
ln: failed to create symbolic link './filelists/miniImagenet/images/images': Read-only file system
ln: failed to create symbolic link './filelists/tieredImagenet/images/tiered_imagenet': Read-only file system
ln: failed to create symbolic link './filelists/dogs/images/Images': Read-only file system
ln: failed to create symbolic link './filelists/aircrafts/images/images': Read-only file system


In [ ]:
!python train.py --dataset dogs --train_aug --resume --resume_wandb_id 11vqc3r9

Checkpoint path: ckpts/dogs/_resnet18_protonet_aug_5way_5shot_16query_lr0.0010
Resuming from wandb ID:  11vqc3r9
wandb: WARNING Tried to auto resume run with id 11vqc3r9 but id 11vqc3r9 is set.
wandb: Currently logged in as: multi-input (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.31 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.10.2
wandb: Run data is saved locally in wandb/run-20210529_072837-11vqc3r9
wandb: Resuming run dry-frost-7
wandb: ⭐️ View project at https://wandb.ai/meta-learners/fsl_ssl
wandb: 🚀 View run at https://wandb.ai/meta-learners/fsl_ssl/runs/11vqc3r9
wandb: Run `wandb off` to turn off syncing.

